In [2]:
# Setup
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import time
import warnings
import torch.nn as nn
import pickle
from transformer_dataset_loader import *
from transformer_utils import *
import pandas as pd
from datasets import Dataset, DatasetDict

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [56]:
dataset_name = 'offensive'
ds = get_ds(dataset_name)

In [57]:
#model_name ='google/electra-small-discriminator'
model_name =  'huawei-noah/TinyBERT_General_4L_312D'
folder_name = 'tinybert'

In [ ]:
set_seed()
model = load_model(model_name)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', num_train_epochs=10)
model.save_pretrained(f'{folder_name}/{dataset_name}/model')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [60]:
def predict_sentences(sentences):
    encoded = [[101] +[tokenizer._convert_token_to_id_with_added_voc(token) for token in tokens] + [102]         
               for tokens in sentences]
    #encoded = tokenizer.encode(sentences, add_special_tokens=True, return_tensors="pt").to(device)
    to_pred = torch.tensor(encoded, device=device)
    outputs = model(to_pred)[0]
    print(outputs)
    return torch.argmax(outputs, dim=1).cpu().numpy()

### save best model's folder as 'model'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(f'{folder_name}/{dataset_name}/model', num_labels=2, torchscript=True).to(device)
model = model.eval()

In [62]:
encoded = tokenizer.encode("i love movies", add_special_tokens=True, return_tensors="pt").to(device)
traced_model = torch.jit.trace(model, encoded)

In [63]:
torch.jit.save(traced_model, f"{folder_name}/{dataset_name}/traced.pt")

In [66]:
predict_sentences([tokenizer.tokenize("i hate movies")])

tensor([[-2.6210,  2.4910]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([1])

In [67]:
predict_sentences([tokenizer.tokenize("i love movies")])

tensor([[ 2.6307, -2.5039]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([0])

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad
count_parameters(model)

In [7]:
# when apply torchscript to models sometimes
torch._C._jit_set_texpr_fuser_enabled(False)